In [1]:
# train_drowsiness_lstm.py
import os, re, json, math, random, shutil
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

In [2]:
# 기본값 설정

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


#### 유형 데이타셋 준비

In [3]:
FEATURE_COLS = [
    "EAR",
    "MAR",
    "yawn_rate_per_min",
    "blink_rate_per_min",
    "avg_blink_dur_sec",
    "longest_eye_closure_sec",
]

# level: -1,0,1,2,3  -> 총 5클래스. (-1 포함)
CLS_LEVELS = [-1, 0, 1, 2, 3]
LEVEL_TO_INDEX = {lvl: i for i, lvl in enumerate(CLS_LEVELS)}
INDEX_TO_LEVEL = {i: lvl for lvl, i in LEVEL_TO_INDEX.items()}

In [4]:
# 데이타셋 구성을 위한 기본 합수들..

def _safe_symlink(src: Path, dst: Path):
    # 숨김 파일(. , ._ 시작)은 무시
    if src.name.startswith(".") or src.name.startswith("._"):
        return

    dst.parent.mkdir(parents=True, exist_ok=True)
    try:
        if dst.exists() or dst.is_symlink():
            dst.unlink()
        os.symlink(src.resolve(), dst)
    except Exception:
        # 윈도우/권한 이슈 대비: 복사로 폴백
        shutil.copy2(src, dst)

def scan_pairs(data_dir: Path) -> List[Tuple[Path, Path]]:
    """
    data_dir 아래의 CSV/JSON 쌍을 찾아 반환.
    파일명(확장자 제외)이 일치하면 쌍으로 간주.
    """
    csvs = {}
    jsons = {}
    for p in data_dir.glob("**/*"):
        if p.is_file():
            stem = p.stem  # 파일명(확장자 제외)
            if p.suffix.lower() == ".csv":
                csvs[stem] = p
            elif p.suffix.lower() == ".json":
                jsons[stem] = p
    pairs = []
    for stem, csv_path in csvs.items():
        if stem in jsons:
            pairs.append((csv_path, jsons[stem]))
    return sorted(pairs)

def split_and_link(pairs: List[Tuple[Path, Path]], out_dir: Path, train_ratio=0.8):
    """
    쌍 목록을 8:2로 분할하고, out_dir/{train,val}에 심볼릭링크(또는 복사) 생성.
    """
    random.shuffle(pairs)
    n_total = len(pairs)
    n_train = int(round(n_total * train_ratio))
    train_pairs = pairs[:n_train]
    val_pairs = pairs[n_train:]

    for split_name, subset in [("train", train_pairs), ("val", val_pairs)]:
        for csv_path, json_path in subset:
            rel = csv_path.name  # 파일명만 사용
            dst_csv = out_dir / split_name / rel
            dst_json = out_dir / split_name / json_path.name
            _safe_symlink(csv_path, dst_csv)
            _safe_symlink(json_path, dst_json)
    return train_pairs, val_pairs

In [ ]:
!pwd
!ls -l ../../../dataset/origin

In [6]:
import os
print(os.getcwd())

/Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/ai-proj3-class5/03.modeling/drowny-model-dongan


In [7]:
# 각자 데이터셋 디렉토리 설정
data_root = "../../../dataset/origin"
prepared_dir = "prepared-dataset"

In [8]:
print("data_root =", data_root)
data_root = Path(data_root).expanduser().resolve()
prep_dir = Path(prepared_dir).resolve()
prep_dir.mkdir(parents=True, exist_ok=True)

data_root = ../../../dataset/origin


In [9]:
# 스캔 & 분할 & 링크
pairs = scan_pairs(data_root)
print(f"총 파일쌍: {len(pairs)}")
train_pairs, val_pairs = split_and_link(pairs, prep_dir, train_ratio=0.8)
print(f"학습: {len(train_pairs)} / 검증: {len(val_pairs)}")

총 파일쌍: 204
학습: 163 / 검증: 41


#### 라벨 관련 함수들

In [10]:
# -------------------------
# 라벨 JSON 해석 (유연 파서)
# -------------------------
def parse_label_json(json_path: Path) -> List[Tuple[int, int, int]]:
    """
    라벨 JSON을 다양한 스키마로 수용하여 [(start, end, level), ...] 리스트로 변환.
    - end는 inclusive로 간주 (start <= frame <= end)
    - 인식 실패 시 빈 리스트
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    segs = []

    def norm_one(d):
        # 키 변형 수용
        if isinstance(d, dict):
            # 유형 1: 명시 키
            s = d.get("start_frame", d.get("frame_start", d.get("start")))
            e = d.get("end_frame", d.get("frame_end", d.get("end")))
            lvl = d.get("level", d.get("lvl", d.get("label", d.get("state"))))
            if s is not None and e is not None and lvl is not None:
                return int(s), int(e), int(lvl)
            # 유형 2: frames: [s, e]
            if "frames" in d and isinstance(d["frames"], (list, tuple)) and len(d["frames"]) >= 2:
                s, e = d["frames"][:2]
                lvl = int(d.get("level", d.get("label", -1)))
                return int(s), int(e), int(lvl)
        return None

    if isinstance(data, list):
        for item in data:
            r = norm_one(item)
            if r: segs.append(r)
    elif isinstance(data, dict):
        # 유형 3: {"segments":[{...}, ...]}
        if "segments" in data and isinstance(data["segments"], list):
            for item in data["segments"]:
                r = norm_one(item)
                if r: segs.append(r)
        # 유형 4: {"ranges": {"100-200": 2, ...}}
        elif "ranges" in data and isinstance(data["ranges"], dict):
            for k, v in data["ranges"].items():
                m = re.match(r"^\s*(\d+)\s*[-:]\s*(\d+)\s*$", str(k))
                if m:
                    s, e = int(m.group(1)), int(m.group(2))
                    segs.append((s, e, int(v)))
        # 유형 5: {"100-200": 1, "201-260": 2}
        else:
            for k, v in data.items():
                m = re.match(r"^\s*(\d+)\s*[-:]\s*(\d+)\s*$", str(k))
                if m:
                    s, e = int(m.group(1)), int(m.group(2))
                    segs.append((s, e, int(v)))

    # 정렬 & 병합은 생략(중복은 뒤값 우선)
    segs.sort(key=lambda x: (x[0], x[1]))
    return segs

def build_frame_labels(max_frame: int, segments: List[Tuple[int,int,int]], default_level=-1) -> np.ndarray:
    """
    0..max_frame 범위에 대해 프레임별 level 배열 만들기.
    """
    labels = np.full((max_frame+1,), default_level, dtype=np.int32)
    for s, e, lvl in segments:
        s = max(0, s); e = min(max_frame, e)
        if e >= s:
            labels[s:e+1] = int(lvl)
    return labels


#### CSV 로딩 관련 함수

In [11]:
def infer_fps_from_csv(df: pd.DataFrame) -> float:
    """
    CSV의 frame과 time_sec로 FPS 추정.
    """
    df = df.sort_values("frame")
    if "time_sec" in df.columns and df["time_sec"].max() > 0:
        f_span = float(df["frame"].iloc[-1] - df["frame"].iloc[0])
        t_span = float(df["time_sec"].iloc[-1] - df["time_sec"].iloc[0])
        if t_span > 0 and f_span > 0:
            return max(1.0, round(f_span / t_span, 2))
    # fallback: frame 차분의 중앙값을 1프레임으로 간주
    return 30.0  # 보수적 기본값

def load_csv(csv_path: Path) -> pd.DataFrame:
    encodings_to_try = ["utf-8", "cp949", "euc-kr"]

    if csv_path.name.startswith(".") or csv_path.name.startswith("._"):
        return pd.DataFrame()  # 빈 데이터프레임 반환 (혹은 None 처리 가능)

    print("current csv_path:", csv_path)
    
    last_err = None
    for enc in encodings_to_try:
        try:
            df = pd.read_csv(csv_path, encoding=enc)
            break
        except UnicodeDecodeError as e:
            last_err = e
    else:
        # 모든 인코딩 실패 시 에러
        raise last_err

    # 필수 컬럼 체크
    must = ["frame", "time_sec"] + FEATURE_COLS
    missing = [c for c in must if c not in df.columns]
    if missing:
        raise ValueError(f"[{csv_path.name}] 누락 컬럼: {missing}")

    df = df.sort_values("frame").reset_index(drop=True)
    return df

#### 윈도윙 함수

In [12]:
def make_windows_for_pair(
    csv_path: Path, json_path: Path,
    window_sec=5.0, hop_sec=1.0,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    하나의 (CSV, JSON) 쌍에서 (X, y) 윈도우 생성
    - X: (num_win, seq_len, num_feat)
    - y: (num_win,)  # 윈도우 대표 레벨 (다수결)
    """
    df = load_csv(csv_path)

    fps = infer_fps_from_csv(df)
    win = max(1, int(round(window_sec * fps)))
    hop = max(1, int(round(hop_sec * fps)))

    # 프레임별 라벨
    segs = parse_label_json(json_path)
    max_frame = int(df["frame"].max())
    frame_labels = build_frame_labels(max_frame, segs, default_level=-1)

    # 특성 행렬
    feats = df[FEATURE_COLS].astype(float).copy()
    # NaN 보정(앞채움→뒤채움→0)
    feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
    feat_np = feats.to_numpy(dtype=np.float32)
    frames = df["frame"].to_numpy(dtype=np.int32)

    # 프레임 인덱스 → 라벨 매핑
    label_by_row = np.array([frame_labels[f] if f <= max_frame else -1 for f in frames], dtype=np.int32)

    X_list, y_list = [], []
    start = 0
    last_start = len(df) - win
    while start <= last_start:
        end = start + win
        seq = feat_np[start:end, :]  # (win, feat)
        seq_labels = label_by_row[start:end]  # (win,)

        # 윈도우 대표 라벨: 다수결(동점이면 마지막 프레임값)
        cnt = Counter(seq_labels.tolist())
        most = max(cnt.items(), key=lambda kv: (kv[1], kv[0]))[0]
        y = most

        X_list.append(seq)
        y_list.append(y)
        start += hop

    if not X_list:
        return np.empty((0, win, len(FEATURE_COLS)), dtype=np.float32), np.empty((0,), dtype=np.int32)

    X = np.stack(X_list, axis=0).astype(np.float32)
    y = np.array(y_list, dtype=np.int32)
    return X, y

#### numpy 데이타셋 만들기

In [13]:
window_sec = 5.0
hop_sec = 1.0 # 0.2 , 혹은 0.0 초 도 테스트 필요..

# 15 초 데이인 경우
# 5 초 기준 1씩 스라이딩시.. 훈련 윈도우 10개 


In [14]:
def build_numpy_dataset(pairs: List[Tuple[Path,Path]], window_sec=5.0, hop_sec=1.0):
    Xs, ys = [], []
    for csv_path, json_path in pairs:
        Xi, yi = make_windows_for_pair(csv_path, json_path, window_sec, hop_sec)
        if len(Xi) > 0:
            Xs.append(Xi); ys.append(yi)
    if not Xs:
        raise RuntimeError("윈도우가 생성되지 않았습니다. 입력 파일/라벨을 확인하세요.")

    max_len = max(X.shape[1] for X in Xs)
    Xs_padded = [
        np.pad(X, ((0,0),(0,max_len-X.shape[1]),(0,0)), mode="constant")
        if X.shape[1] < max_len else X
        for X in Xs
    ]
    
    X = np.concatenate(Xs_padded, axis=0)
    y = np.concatenate(ys, axis=0)
    return X, y

In [15]:
# 넘파이 데이터셋
Xtr, ytr = build_numpy_dataset(train_pairs, window_sec, hop_sec)
Xva, yva = build_numpy_dataset(val_pairs, window_sec, hop_sec)

current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/20-FemaleNoGlasses-Yawning-labeled.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/27-MaleGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/10-MaleNoGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/12-MaleGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/19-FemaleNoGlasses-Normal-labeled.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/43-MaleNoGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/

/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with '

current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/32-MaleGlasses-Yawning-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/27-MaleGlasses-Yawning-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/36-MaleNoGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/5-FemaleGlasses-Yawning-labeled.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/17-MaleNoGlasses-Yawning-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/35-MaleNoGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/orig

/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with '

current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/7-FemaleGlasses-Normal-labeled.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/43-MaleNoGlasses-Yawning-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/15-MaleNoGlasses-Yawning-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/8-MaleGlassesBeard-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/40-MaleNoGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/origin/26-MaleNoGlasses-Normal-label.csv
current csv_path: /Users/dongango/Documents/dongango/2025/교육/EastSoft-와썹 AI 모델 개발/work/dataset/

/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  feats = feats.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
/var/folders/5k/zrsrnd7s18b7z_zm0b5yktj40000gn/T/ipykernel_73359/2111207534.py:24: FutureWarning: DataFrame.fillna with '

In [14]:
train_pairs

[(PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/20-FemaleNoGlasses-Yawning-labeled.csv'),
  PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/20-FemaleNoGlasses-Yawning-labeled.json')),
 (PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/27-MaleGlasses-Normal-label.csv'),
  PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/27-MaleGlasses-Normal-label.json')),
 (PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/10-MaleNoGlasses-Normal-label.csv'),
  PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/10-MaleNoGlasses-Normal-label.json')),
 (PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/12-MaleGlasses-Normal-label.csv'),
  PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개발/dataset/origin/12-MaleGlasses-Normal-label.json')),
 (PosixPath('/Volumes/EX_SSD_1T/와섭 AI 프로젝트/3차/40.개바

#### 스케일링..

In [16]:
def fit_standardizer(X: np.ndarray):
    """
    채널별 표준화 스케일러(평균/표준편차) 반환.
    """
    # X shape: (N, T, C)
    mean = X.reshape(-1, X.shape[-1]).mean(axis=0)
    std = X.reshape(-1, X.shape[-1]).std(axis=0)
    std = np.where(std < 1e-8, 1.0, std)
    return mean.astype(np.float32), std.astype(np.float32)

def apply_standardizer(X: np.ndarray, mean: np.ndarray, std: np.ndarray):
    return (X - mean) / std

In [17]:
# 스케일링 (학습 기반)
mean, std = fit_standardizer(Xtr)
Xtr = apply_standardizer(Xtr, mean, std)
Xva = apply_standardizer(Xva, mean, std)

#### TF 데이타셋

In [18]:
def to_tf_dataset(X: np.ndarray, y: np.ndarray, batch=64, shuffle=True):
    # y를 인덱스로 변환(-1..3 → 0..4)
    y_idx = np.array([LEVEL_TO_INDEX[int(v)] for v in y], dtype=np.int32)
    ds = tf.data.Dataset.from_tensor_slices((X, y_idx))
    if shuffle:
        ds = ds.shuffle(min(len(X), 10000), seed=RANDOM_SEED)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

In [19]:
batch = 64

In [20]:
# TF Dataset
ds_tr = to_tf_dataset(Xtr, ytr, batch=batch, shuffle=True)
ds_va = to_tf_dataset(Xva, yva, batch=batch, shuffle=False)

I0000 00:00:1756391603.091287 12234791 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1756391603.091315 12234791 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### 모델 생성 함수

In [21]:
# =========================
# 2) 모델 --> 테스트 시 여러 모델 테스트 필요 함..
# =========================
def build_lstm_model(seq_len: int, n_feat: int, n_classes: int = 5) -> tf.keras.Model:
    """
    BiLSTM 스택 + LayerNorm + Dropout.
    합리적 기본값으로 튜닝된 구조.
    """
    inp = layers.Input(shape=(seq_len, n_feat))
    x = layers.LayerNormalization()(inp)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(n_classes, activation="softmax")(x)

    model = models.Model(inp, out)
    model.compile(
        optimizer=optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [22]:
# 모델
seq_len, n_feat = Xtr.shape[1], Xtr.shape[2]
model = build_lstm_model(seq_len, n_feat, n_classes=len(CLS_LEVELS))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 6)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 150, 6)         │            12 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 150, 256)       │       138,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 150, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 311,185 (1.19 MB)

 Trainable params: 311,185 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

### 학습.

In [23]:
# 클래스 불균형 가중치(선택)
counts = Counter([LEVEL_TO_INDEX[int(v)] for v in ytr])
total = sum(counts.values())
class_weight = {i: total / (len(counts) * counts[i]) for i in counts}
print("class_weight:", class_weight)

class_weight: {2: 1.490357142857143, 3: 1.671875, 4: 0.43723805532271587, 1: 2.2532397408207343}


In [24]:
# 콜백
cbs = [
    callbacks.ModelCheckpoint("best_lstm.keras", monitor="val_loss", save_best_only=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3),
    callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True),
]

In [25]:
# 학습
history = model.fit(
    ds_tr,
    validation_data=ds_va,
    epochs=30,
    class_weight=class_weight,
    callbacks=cbs,
    verbose=1,
)

Epoch 1/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 24s 252ms/step - accuracy: 0.5705 - loss: 1.1097 - val_accuracy: 0.7409 - val_loss: 0.6358 - learning_rate: 0.0010
Epoch 2/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 213ms/step - accuracy: 0.7374 - loss: 0.7366 - val_accuracy: 0.7095 - val_loss: 0.8433 - learning_rate: 0.0010
Epoch 3/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 218ms/step - accuracy: 0.7610 - loss: 0.7120 - val_accuracy: 0.7758 - val_loss: 0.5934 - learning_rate: 0.0010
Epoch 4/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 212ms/step - accuracy: 0.7716 - loss: 0.6522 - val_accuracy: 0.7840 - val_loss: 0.5647 - learning_rate: 0.0010
Epoch 5/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 211ms/step - accuracy: 0.7712 - loss: 0.5970 - val_accuracy: 0.7321 - val_loss: 0.6318 - learning_rate: 0.0010
Epoch 6/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 211ms/step - accuracy: 0.7563 - loss: 0.6268 - val_accuracy: 0.8052 - val_loss: 0.4740 - learning_rate: 0.0010
Epoch 7/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 14s 216ms/step - accuracy: 0.7940 - loss: 0.

#### 평가

In [26]:
# 평가
eval_res = model.evaluate(ds_va, verbose=0)
print(f"[VAL] loss={eval_res[0]:.4f}, acc={eval_res[1]:.4f}")

[VAL] loss=0.3764, acc=0.8585


In [27]:
# 예측 샘플 & 혼동행렬
y_true = np.array([y for _, y in ds_va.unbatch().as_numpy_iterator()])
# y_true = np.concatenate([y for _, y in ds_va.unbatch().as_numpy_iterator()], axis=0)
y_pred = []
for xb, _ in ds_va:
    pb = model.predict(xb, verbose=0)
    y_pred.append(np.argmax(pb, axis=1))
    # y_pred.append(np.argmax(pb.numpy(), axis=1))
y_pred = np.concatenate(y_pred, axis=0)

# 간단 리포트
from sklearn.metrics import classification_report, confusion_matrix
labels = sorted(set(y_true) | set(y_pred))  # 실제 등장한 클래스
print(classification_report(
    y_true, y_pred,
    labels=labels,
    target_names=[str(INDEX_TO_LEVEL[i]) for i in labels]
))

#print(classification_report(y_true, y_pred, target_names=[str(INDEX_TO_LEVEL[i]) for i in range(len(CLS_LEVELS))]))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       179
           1       0.85      0.83      0.84       282
           2       0.72      0.72      0.72       271
           3       0.91      0.91      0.91       731

    accuracy                           0.86      1463
   macro avg       0.84      0.84      0.84      1463
weighted avg       0.86      0.86      0.86      1463

Confusion matrix:
 [[162   8   9   0]
 [ 11 235  14  22]
 [  4  24 196  47]
 [  3  11  54 663]]


In [28]:
# 산출물 저장
np.savez("scaler_mean_std.npz", mean=mean, std=std)
with open("feature_cols.txt", "w") as f:
    f.write("\n".join(FEATURE_COLS))
print("✔ 저장: best_lstm.keras, scaler_mean_std.npz, feature_cols.txt")

✔ 저장: best_lstm.keras, scaler_mean_std.npz, feature_cols.txt


In [30]:
# === artifacts_io.py ===
from __future__ import annotations
import json, pickle
from pathlib import Path
from typing import Any, Dict

def save_model_artifacts(
    export_dir: str | Path,
    tf_model,                    # tf.keras.Model
    scaler=None,                 # e.g., sklearn.preprocessing.StandardScaler
    label_encoder=None,          # e.g., sklearn.preprocessing.LabelEncoder
    meta: Dict[str, Any] | None = None
):
    export_dir = Path(export_dir)
    export_dir.mkdir(parents=True, exist_ok=True)

    # 1) TF 모델 저장 (SavedModel 형식 권장)
    model_dir = export_dir / "saved_model"
    tf_model.save(model_dir)  # creates saved_model.pb + variables/

    # 2) 전처리기 저장
    if scaler is not None:
        with open(export_dir / "scaler.pkl", "wb") as f:
            pickle.dump(scaler, f)
    if label_encoder is not None:
        with open(export_dir / "label_encoder.pkl", "wb") as f:
            pickle.dump(label_encoder, f)

    # 3) 메타데이터 저장 (필수 하이퍼파라미터/특성 순서/윈도우 크기 등)
    meta = dict(meta or {})
    # 예: 모델이 기대하는 입력 특성명/순서 & 윈도우 길이
    meta.setdefault("feature_order", [
        "EAR", "MAR",
        "yawn_rate_per_min", "blink_rate_per_min",
        "avg_blink_dur_sec", "longest_eye_closure_sec",
    ])
    meta.setdefault("seq_len", 50)  # 예: 30fps 기준 ~1.6초/ 또는 5초 윈도(=150) 등
    meta.setdefault("fps", 30)
    meta.setdefault("window_sec", 5.0)
    meta.setdefault("hop_sec", 1.0)
    with open(export_dir / "meta.json", "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"[save] Artifacts saved to: {export_dir}")


In [ ]:
# ... 모델 학습 완료 후 ...
export_dir = "./export/zoligima_lstm_v1"
meta = {
    "feature_order": [
        "EAR",
        "MAR",
        "yawn_rate_per_min",
        "blink_rate_per_min",
        "avg_blink_dur_sec",
        "longest_eye_closure_sec"
   ],
    "seq_len": 150,                # 학습에 사용한 시퀀스 길이
    "fps": 30,
    "window_sec": window_sec,
    "hop_sec": hop_sec,
    "num_classes": 5,
    "model_type": "LSTM",
}
save_model_artifacts(export_dir, model, None, None, meta)

In [41]:
%run ./train_artifacts_patch.py



In [43]:
# from train_artifacts_patch import save_artifacts

save_artifacts(
    model,                          # tf.keras 모델
    scaler=None,          # StandardScaler 등 (없으면 None)
    feature_names=[
        "EAR","MAR","yawn_rate_per_min","blink_rate_per_min",
        "avg_blink_dur_sec","longest_eye_closure_sec"
    ],                              # 모델이 기대하는 feature 순서
    meta=dict(
        project="drowny",
        version="v1",
        window_sec=5.0,             # 모델이 학습에 사용한 윈도 길이(초)
        hop_sec=1.0,                # 슬라이딩 간격(초)
        sampling_hz=30.0,           # 초당 샘플 수(프레임/메시지)
        # seq_len은 자동 추정. 불명확하면 여기서 직접 지정 가능
        label_map={
            "0":"awake","1":"early_drowsy","2":"drowsy","3":"very_drowsy"
        }
    ),
    out_dir="artifacts/drowny-v1"
)

[save_artifacts] Saved model -> artifacts/drowny-v1/model.keras
[save_artifacts] Saved meta -> artifacts/drowny-v1/meta.json
[save_artifacts] DONE.


### 예측 코드 작성

In [57]:
import json
import numpy as np
from collections import deque
from confluent_kafka import Consumer, Producer
import tensorflow as tf

In [48]:
# =============================
# 1. Kafka 설정
# =============================
kafka_conf = {
    "bootstrap.servers": "kafka.dongango.com:9094",
    "group.id": "drowny-lstm-consumer-test",
    "auto.offset.reset": "earliest"
}

input_topic = "sess-9kt6byvo-mev3xjib"
output_topic = input_topic + "-LSTM"

# level: -1,0,1,2,3  -> 총 5클래스. (-1 포함)
CLS_LEVELS = [-1, 0, 1, 2, 3]

In [49]:
FEATURE_COLS = ["EAR", "MAR", "yawn_rate_per_min", "blink_rate_per_min",
                "avg_blink_dur_sec", "longest_eye_closure_sec"]

SEQ_LEN = 150   # 5초 @ 30fps
HOP = 30        # 1초 단위 슬라이딩

In [50]:
# =============================
# 3. Kafka Consumer / Producer 초기화
# =============================
consumer = Consumer(kafka_conf)
consumer.subscribe([input_topic])

producer = Producer({"bootstrap.servers": kafka_conf["bootstrap.servers"]})

In [51]:
# =============================
# 4. 윈도우 버퍼 (deque 사용)
# =============================
window = deque(maxlen=SEQ_LEN)
frame_numbers = deque(maxlen=SEQ_LEN)

def preprocess_value(value_dict):
    """필요한 feature만 뽑아 float 배열로 변환"""
    return [float(value_dict.get(col, np.nan)) for col in FEATURE_COLS]

def predict_and_publish():
    """버퍼에서 예측 후 Kafka publish"""
    if len(window) == SEQ_LEN:
        X = np.array(window).reshape(1, SEQ_LEN, len(FEATURE_COLS))
        
        # ✅ 표준화 적용
        X = apply_standardizer(X, mean, std)

        y_pred = model.predict(X, verbose=0)
        class_index = int(np.argmax(y_pred, axis=1)[0])   # multi-class softmax 결과
        drowny_level = CLS_LEVELS[class_index]
        
        result = {
            "frame": int(frame_numbers[-1]),  # 마지막 frame 번호
            "drowny-level": drowny_level
        }
        
        producer.produce(
            output_topic,
            key="drowny-lstm-result",
            value=json.dumps(result)
        )
        producer.flush()
        print("Published:", result)

In [55]:
def transform_json(input_json: dict) -> dict:
    # timestamp_ms -> time_sec 변경
    output = {
        "frame": input_json.get("frame"),
        "time_sec": input_json.get("timestamp_ms"),
    }

    # metrics 값 병합
    metrics = input_json.get("metrics", {})
    for k, v in metrics.items():
        if k == "mar":  # "mar" 키 제외
            continue
        # NaN 값 0으로 치환
        if isinstance(v, float) and math.isnan(v):
            output[k] = 0.0
        else:
            output[k] = v

    return output

In [56]:
# =============================
# 5. 스트리밍 loop
# =============================
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print("Kafka error:", msg.error())
            continue

        #value = json.loads(msg.value().decode("utf-8"))
        value = transform_json(json.loads(msg.value().decode("utf-8")))
        frame_no = int(value.get("frame"))

        # 순서 체크 (frame 번호가 이전보다 작으면 skip)
        if frame_numbers and frame_no <= frame_numbers[-1]:
            continue
        
        print(value)
        # feature 추출 및 버퍼에 저장
        features = preprocess_value(value)
        window.append(features)
        frame_numbers.append(frame_no)

        # 윈도우가 다 차면 예측
        if len(window) == SEQ_LEN:
            predict_and_publish()

            # 1초(30 frame)씩 슬라이딩 → 앞부분 drop
            for _ in range(HOP):
                if window:
                    window.popleft()
                    frame_numbers.popleft()

except KeyboardInterrupt:
    print("Stopped by user")
finally:
    consumer.close()

RuntimeError: Consumer closed